#### Задание 1.

Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = [‘python’, ‘парсинг’]

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [1]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [2]:
res = requests.get('https://habr.com/ru/')
res

<Response [200]>

In [3]:
req = requests.get('https://habr.com/ru/')
soup = BeautifulSoup(req.text, 'html.parser')

In [4]:
posts = soup.find_all('article', class_='post')
len(posts)

20

In [5]:
import re

In [6]:
keywords = ['Raspberry ', 'Гаджеты', 'python', 'JavaScript', 'Node.JS', 'Управление проектами', 'alkaline']

In [7]:
my_posts = {}
my_keywords = []
for post in posts:
    for keyword in keywords:
        if re.findall(keyword, post.text):
            my_keywords.append(keyword)
            post_id = post.parent.attrs.get('id')
            post_id = int(post_id.split('_')[-1])
            my_posts.setdefault(post_id, my_keywords)    
    my_keywords = []     
print(my_posts)

{510904: ['JavaScript'], 511280: ['Управление проектами']}


In [8]:
my_posts = {}
my_keywords = []
for post in posts:
    for keyword in keywords:
        if re.findall(keyword, post.text):
            my_keywords.append(keyword)
            post_link = post.find('a', class_='post__title_link').get('href')
            my_posts.setdefault(post_link, my_keywords)    
    my_keywords = []     
print(my_posts)

{'https://habr.com/ru/company/ruvds/blog/510904/': ['JavaScript'], 'https://habr.com/ru/company/oleg-bunin/blog/511280/': ['Управление проектами']}


In [9]:
results = []
result = []
for post in posts:
    for keyword in keywords:
        if re.findall(keyword, post.text):
            post_date = post.find('span', class_='post__time').get_text()
            post_header = post.find('a', class_='post__title_link').get_text()
            post_link = post.find('a', class_='post__title_link').get('href')
            result = [post_date, post_header, post_link]
            results.append(result)
print(results)

[['сегодня в 16:17', 'Wasm или не Wasm?', 'https://habr.com/ru/company/ruvds/blog/510904/'], ['сегодня в 15:20', 'Почему онлайн-конференция по управлению знаниями — это не скучно', 'https://habr.com/ru/company/oleg-bunin/blog/511280/']]


In [10]:
df = pd.DataFrame(results, columns=['Дата', 'Заголовок', 'Ссылка'])
df

,Дата,Заголовок,Ссылка
0,сегодня в 16:17,Wasm или не Wasm?,https://habr.com/ru/company/ruvds/blog/510904/
1,сегодня в 15:20,Почему онлайн-конференция по управлению знания...,https://habr.com/ru/company/oleg-bunin/blog/51...


#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

#### Алгоритм
1. Нахожу ссылку на каждый пост на главной странице (всего 20)

2. Открываю каждую и ищу вхождения из keywords. 

3. Если нашёл, то добавляю в result.

4. Результирующий список results.append(results)

In [11]:
keywords = ['Софт', 'Физика']

In [12]:
results = []
result = []
for post in posts:
    post_link = post.find('a', class_='post__title_link').get('href')
    post_content = BeautifulSoup(requests.get(post_link).text, 'html.parser')
    for keyword in keywords:
        if re.findall(keyword, post_content.text):
            post_date = post_content.find('span', class_='post__time').get_text()
            post_header = post_content.find('span', class_='post__title-text').get_text()
            post_text = post_content.find('div', class_='post__text').get_text()
            result = [post_date, post_header, post_link, post_text]
            results.append(result)
len(results)

# post_tag.text

2

In [13]:
df = pd.DataFrame(results, columns=['Дата', 'Заголовок', 'Ссылка', 'Текст статьи'])
df

,Дата,Заголовок,Ссылка,Текст статьи
0,сегодня в 14:26,Расследование: создатель AlterOffice украл код...,https://habr.com/ru/post/494070/,\n\r\nПолгода назад я опубликовал на Хабре ста...
1,вчера в 21:04,Лекция с разбором правды и мифов об аварии на ...,https://habr.com/ru/post/511156/,Внимательные читатели и подписчики наверняка ж...
